In [ ]:
%matplotlib inline

import json
import pickle
import pandas
import numpy
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit

from pandas import DataFrame


def log_as_dataframe(log):
    columns = ['timestamp', 'level', 'source', 'function', 'data']
    df = pandas.DataFrame(log, columns=columns)
    return df.set_index('timestamp').sort_index()


df = log_as_dataframe(pickle.load(open('log.pkl', 'rb')))
df = df[df['function'] == 'log_front_sensors_calibration']
df = DataFrame([json.loads(x.replace('nan', 'NaN')) for x in df['data'].values])
df['left_raw'] = df['left_raw_on'] - df['left_raw_off']
df['right_raw'] = df['right_raw_on'] - df['right_raw_off']
df['micrometers'] = df['micrometers'] - df['micrometers'][0]
df['micrometers'] = (180 * 2 - 12 - 36) * 1000 - df['micrometers']
df = df[(df['micrometers'] > 80000) & (df['micrometers'] < 270000)]
df['meters'] = df['micrometers'] / 1000000
df = df.set_index('meters')
df = df.sort_index(ascending=True)
df.head()

In [ ]:
df[['left_raw', 'right_raw']].plot()

In [ ]:
popt_FR, pcov_FR = curve_fit(lambda t, a, b: numpy.exp(a / (b + t)),
                             df.index.values,
                             df['right_raw'].values)
popt_FL, pcov_FL = curve_fit(lambda t, a, b: numpy.exp(a / (b + t)),
                             df.index.values, 
                             df['left_raw'].values)

In [ ]:
def generate_curve(x, parameters):
    return numpy.exp(parameters[0] / (parameters[1] + df.index.values))


f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
ax1.plot(df.index.values, df['right_raw'])
ax1.plot(df.index.values, generate_curve(df.index.values, popt_FR))
ax2.plot(df.index.values, df['left_raw'])
ax2.plot(df.index.values, generate_curve(df.index.values, popt_FL))
plt.show()

print('FR_a {}, FR_b {}'.format(popt_FR[0],popt_FR[1]))
print('FL_a {}, FL_b {}'.format(popt_FL[0],popt_FL[1]))